In [31]:
# %pip install torch torchvision torchaudio

In [32]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet50

In [33]:
INPUT_DIR_A = '../data/model_input_a'

In [34]:
import torch
from torchvision import datasets, transforms

# Define the transformation
transform = transforms.Compose([
    transforms.RandomRotation(10),  # Apply random rotation with a maximum angle of 10 degrees
    transforms.ToTensor()  # Convert the image to a tensor
])

# Define the dataset
dataset = datasets.ImageFolder(INPUT_DIR_A, transform=transform)

# Verify the classes
classes = dataset.classes
print(classes)


['Monet', 'Picasso']


In [35]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f"Device: {device}")

# Load the pre-trained ResNet model
model = resnet50(pretrained=True)

# Freeze all the parameters in the pre-trained model
for param in model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to match the number of classes in your dataset
num_classes = len(classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

# Define the number of training epochs
num_epochs = 10

# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    
    # Set the model to training mode
    model.train()
    
    for images, labels in dataset:
        # Move the images and labels to the GPU if available
        images = images.to(device)
        labels = labels.to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        
        # Compute the loss
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer.step()
        
        # Update the running loss
        running_loss += loss.item()
        
        # Compute the predicted labels
        _, predicted = torch.max(outputs.data, 1)
        
        # Update the total and correct predictions
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    # Compute the accuracy
    accuracy = 100 * correct / total
    
    # Print the loss and accuracy for each epoch
    print(f"Epoch {epoch+1}/{num_epochs}: Loss = {running_loss:.4f}, Accuracy = {accuracy:.2f}%")


Device: mps


/Users/joshuabensemann/Documents/GitHub/Behavioral-Bits-and-Bytes/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/joshuabensemann/Documents/GitHub/Behavioral-Bits-and-Bytes/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /Users/joshuabensemann/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100.0%


AttributeError: 'int' object has no attribute 'to'